## 第一部分：PyTorch深度学习基础知识


### 2.2深度学习在实现上的特殊性
- 样本量大，通常需要分批（bath）加载
- 逐层、模板化搭建网络（卷积层、全连接、LSTM等）
- 多样化的损失函数和优化器设计*
- GPU的使用
- 模块间配合

## 2.3PyTorch深度学习模块
- 世纪使用根据需要进行修改
- 深度学习：搭积木
  - 2.3.1基本配置
  - 2.3.2数据读入
  - 2.3.3模型构建
  - 2.3.4损失函数
  - 2.3.5优化器
  - 2.3.6优化与评估
- 边学边练，通过实战案例巩固
- 任务：FashionMNIST时装分类
- 数据简介：
  - 10类图片
  - 32*32px

### 2.
- 导入必要的packages
 - os ,numpy, pandas, torch, torch.nn,torch.optin, torch.utils.data
- 配置训练过程中的超参数
 - batch size,learning rate,max_epochs,num_workers
- 配置训练用的硬件设备
 - cpu ,GPU,
- 代码演示

In [1]:
#导包
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.optim as optimizer

#### 配置训练环境和超参数


In [2]:
# 配置GPU,这里有两种方式
## 一：os.environ
#os.environ['CUDA_VISIBLE_DEVICES']='0'#这里的0就是第一块显卡
# 二：使用“device”,后续对要使用GPU的变量.to(device)即可
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

## 配置其他超参数，batch size,learning rate,max_epochs,num_workers
batch_size=256#每次训练数据的量
num_workers=4#有多少个线程
lr=1e-4#learn rate 每次更新步长
epochs=20#总的epochs 训练多少轮


In [3]:
#torch.cuda.is_available()#我没有显卡，呜呜呜

- PyTorch数据读入是通过Dataset+DataLoader的方式完成的，Dataset定义好数据的格式和数据变换形式，DataLoader用iterative的方式不断读入批次数据。

  我们可以定义自己的Dataset类来实现灵活的数据读取，定义的类需要继承PyTorch自身的Dataset类。主要包含三个函数：

  - __init__: 用于向类中传入外部参数，同时定义样本集

  - __getitem__: 用于逐个读取样本集合中的元素，可以进行一定的变换，并将返回训练/验证所需的数据

  - __len__: 用于返回数据集的样本数

   下面以cifar10数据集为例给出构建Dataset类的方式：

In [16]:
from torchvision import transforms
image_size=28
data_transform=transforms.Compose([
   # transforms.ToPILImage(),
    transforms.Resize(image_size),
    transforms.ToTensor()
])

In [32]:
#读取方式一
from torchvision import datasets
train_data = datasets.FashionMNIST(root= './',train=True,download=True,transform=data_transform)#train_data是训练集
test_data = datasets.FashionMNIST(root='./',train=False,download=True,transform=data_transform)#test_data是测试

In [ ]:
#读取方式二:读入csv格式的数据，自行构建Dataset类
#csv数据下载链接：https://www.kaggle.com/zalando-research/fashionmnist
class FMDataset(Dataset):
    def __int__(self,df,transform=None):
        self.df=df
        self.transform=transform
        self.images=df.iloc[:,1:].values.astype(np.uint8)
        self.labels=df.iloc[:,0].values
    def __len__(self):
        return len(self.images)
    def __getitem__(self,idx):
        image=self.images[idx].reshape(28,28,1)
        label=int(self.labels[idx])
        if self.transform is not None:
            image=self.transform(image)  
        else:
            image=torch.tensor(image/255.,dtype=torch.float)
        label=torch.tensor(label,dtype=torch.long)
        return image,label
train_df =pd.read_csv('./FashionMNIST/fashion-mnist_train.csv')
test_df =pd.read_csv('./FashionMNIST/fashion-mnist_test.csv')
train_data=FMDataset(train_df,data_transform)
test_data=FMDataset(test_df,data_transform)

In [ ]:
train.df

In [ ]:
#在构建训练和测试集完成后，需要定义DataLoader类，以使训练和测试时加载数据